In [138]:
import numpy as np
import sympy as sp

In [169]:
def R(x, lower, upper):
    if np.sum(lower) > x or np.sum(upper) < x:
        raise ValueError('infeasible configuration')

    solution = lower.copy()
    x = x - np.sum(lower)

    idx = 0
    while x > 0 and idx < len(lower):
        if upper[idx] - lower[idx] < x:
            solution[idx] = upper[idx]
            x -= upper[idx] - lower[idx]
        else:
            solution[idx] = solution[idx] + x
            x = 0.0
        idx += 1
    
    return solution

In [196]:
lower = np.array([0.3, 0.4, 0.6])
upper = np.array([0.75, 0.8, 0.9])
x = 0.8 * len(lower)

In [197]:
R(x, lower, upper)

array([0.75, 0.8 , 0.85])

In [198]:
1 - R(3 - x, 1 - np.flip(upper), 1 - np.flip(lower))

array([0.85, 0.8 , 0.75])

In [158]:
p = sp.Symbol('p')
n = sp.Symbol('n')
fpr = sp.Symbol('fpr')
tpr = sp.Symbol('tpr')
acc = sp.Symbol('acc')
auc = sp.Symbol('auc')

auc0 = sp.solve(1 - fpr*(1 - tpr) - auc, tpr)[0]
acc = (p * tpr + n * (1 - fpr))/(p + n)

In [162]:
tmp.subs({fpr: 1})

auc*p/(n + p)

In [163]:
tmp.subs({fpr: 1 - auc})

auc*n/(n + p)

In [160]:
tmp = acc.subs({tpr: auc0})

In [155]:
sp.solve(sp.diff(tmp, fpr), fpr)

[-sqrt(p/n), sqrt(p/n)]

In [156]:
sp.diff(sp.diff(tmp, fpr), fpr)

(-2*p/fpr**2 + 2*p*(fpr - 1)/fpr**3)/(n + p)

In [86]:
p = 50
n = 120

def acc(auc):
    return float(1 - np.sqrt(2*(1 - auc)*p*n)/(p + n)), 1 - min(p, n)**2/(2*p*n), max(p, n)/(p + n)

In [87]:
acc(0.79166)

(0.70587764709647, 0.7916666666666666, 0.7058823529411765)

In [127]:
ps = np.array([17, 22, 19])
ns = np.array([45, 41, 44])
k = 3

avg_auc = 0.8

In [128]:
def avg_acc_cmax(ps, ns, aucs):
    def acc_cmax(p, n, auc):
        return (max(p, n) + min(p, n)*np.sqrt(2*auc - 1))/(p + n)
    return np.mean([acc_cmax(p, n, auc) for p, n, auc in zip(ps, ns, aucs)])

In [129]:
avg_acc_cmax(ps, ns, np.array([0.8, 0.86, 0.79]))

np.float64(0.9377985564153045)

In [164]:
def avg_acc_cmax_kfold(ps, ns, avg_auc):
    w = np.array([min(p, n)/(p + n) for p, n in zip(ps, ns)])
    c = (avg_auc - 0.5) * 2 * len(ps)
    z = w / np.linalg.norm(w) * np.sqrt(c)
    aucs = (z**2 + 1)/2

    print(aucs)

    return avg_acc_cmax(ps, ns, aucs)

In [166]:
avg_acc_cmax_kfold([1, 10, 100], [100, 10, 1], np.mean([0.8, 0.86, 0.79]))

[0.50037222 1.44925556 0.50037222]


np.float64(1.0565568844710629)

In [132]:
import pulp as pl

In [133]:
def avg_acc_max_kfold(ps, ns, avg_auc):
    problem = pl.LpProblem('acc')
    
    aucs = [pl.LpVariable(f'auc{idx}') for idx in range(len(ps))]
    ws = [min(p, n)/(p + n) for p, n in zip(ps, ns)]
    maxs = [max(p, n)/(p + n) for p, n in zip(ps, ns)]

    for auc in aucs:
        problem += auc >= 0.0
        problem += auc <= 1.0
    
    problem += sum(aucs)/len(ps) == avg_auc

    problem += -sum(w*auc for w, auc in zip(ws, aucs))

    problem.solve()

    aucs = [variable.varValue for variable in problem.variables()]

    print(ws)
    print(aucs)

    return np.mean([ma + w*auc for ma, w, auc in zip(maxs, ws, aucs)])

In [134]:
avg_acc_max_kfold(ps, ns, 0.7)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gykovacs/anaconda3/envs/mlscorecheck/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/c64ab868c2e44521a1e5082918e0a7d0-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/c64ab868c2e44521a1e5082918e0a7d0-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 25 RHS
At line 33 BOUNDS
At line 37 ENDATA
Problem MODEL has 7 rows, 3 columns and 9 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 1 (-6) rows, 3 (0) columns and 3 (-6) elements
0  Obj 0 Primal inf 2.099999 (1) Dual inf 0.9249842 (3)
1  Obj -0.67821301
Optimal - objective value -0.67821301
After Postsolve, objective -0.67821301, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -0.6782130056 - 1 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total 

np.float64(0.9177419354838711)

In [137]:
problem = avg_acc_max_kfold(ps, ns, 0.6)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gykovacs/anaconda3/envs/mlscorecheck/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/5aad602a21c84544a223dbcc80700280-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/5aad602a21c84544a223dbcc80700280-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 25 RHS
At line 33 BOUNDS
At line 37 ENDATA
Problem MODEL has 7 rows, 3 columns and 9 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 1 (-6) rows, 3 (0) columns and 3 (-6) elements
0  Obj 0 Primal inf 1.799999 (1) Dual inf 0.9249842 (3)
1  Obj -0.59047619
Optimal - objective value -0.59047619
After Postsolve, objective -0.59047619, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -0.5904761905 - 1 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total 

In [68]:
problem.variables()

[auc0, auc1, auc2]

In [69]:
problem

acc:
MINIMIZE
-0.3333333333333333*auc0 + -0.3333333333333333*auc1 + -0.3333333333333333*auc2 + -0.0
SUBJECT TO
_C1: auc0 >= 0

_C2: auc0 <= 1

_C3: auc1 >= 0

_C4: auc1 <= 1

_C5: auc2 >= 0

_C6: auc2 <= 1

_C7: 0.333333333333 auc0 + 0.333333333333 auc1 + 0.333333333333 auc2 = 0.8

VARIABLES
auc0 free Continuous
auc1 free Continuous
auc2 free Continuous

In [72]:
problem.variables()[2].varValue

0.4

In [50]:
solution = problem.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gykovacs/anaconda3/envs/mlscorecheck/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/76c29bb6f1eb4e40add8ac64a68304c4-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/76c29bb6f1eb4e40add8ac64a68304c4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 25 RHS
At line 33 BOUNDS
At line 37 ENDATA
Problem MODEL has 7 rows, 3 columns and 9 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-7) rows, 0 (-3) columns and 0 (-9) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value -0.26666667
After Postsolve, objective -0.26666667, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -0.2666666667 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):      